This version:

Notation: $y_{i}$ is a scalar, and $x_{i}$ is a $K\times1$ vector. $Y$
is an $n\times1$ vector, and $X$ is an $n\times K$ matrix.

Algebra of Least Squares
========================

OLS estimator
-------------

As we have learned from the linear project model, the parameter $\beta$
$$\begin{aligned}
y_{i} & = & x'_{i}\beta+e_{i}\\
E[x_{i}e_{i}] & = & 0\end{aligned}$$ can be written as
$\beta=\left(E\left[x_{i}x_{i}'\right]\right)^{-1}E\left[x_{i}y_{i}\right].$

While population is something imaginary, in reality we possess a sample
of $n$ observations. We thus replace the population mean
$E\left[\cdot\right]$ by the sample mean, and the resulting estimator is
$$\widehat{\beta}=\left(\frac{1}{n}\sum_{i=1}^{n}x_{i}x_{i}'\right)^{-1}\frac{1}{n}\sum_{i=1}^{n}x_{i}y_{i}=\left(X'X\right)^{-1}X'y.$$
This is one way to motivate the OLS estimator.

Alternatively, we can derive the OLS estimator from minimizing the sum
of squared residuals
$$Q\left(\beta\right)=\sum_{i=1}^{n}\left(y_{i}-x_{i}'\beta\right)^{2}=\left(Y-X\beta\right)'\left(Y-X\beta\right).$$
By the first-order condition
$$\frac{\partial}{\partial\beta}Q\left(\beta\right)=-2X'\left(Y-X\beta\right),$$
the optimality condition gives exactly the same $\widehat{\beta}$.
Moreover, the second-order condition
$$\frac{\partial^{2}}{\partial\beta\partial\beta'}Q\left(\beta\right)=2X'X$$
shows that $Q\left(\beta\right)$ is convex in $\beta$.
($Q\left(\beta\right)$ is strictly convex in $\beta$ if $X'X$ is
positive definite.)

Here we introduce some definitions and properties in OLS estimation.

-   Fitted value: $\widehat{Y}=X\widehat{\beta}$.

-   Projector: $P_{X}=X\left(X'X\right)^{-1}X$; Annihilator:
    $M_{X}=I_{n}-P_{X}$.

-   $P_{X}M_{X}=M_{X}P_{X}=0$.

-   If $AA=A$, we call it an idempotent matrix. Both $P_{X}$ and $M_{X}$
    are idempotent.

-   Residual:
    $\widehat{e}=Y-\widehat{Y}=Y-X\widehat{\beta}=M_{X}Y=M_{X}\left(X\beta+e\right)=M_{X}e$.

-   $X'\widehat{e}=XM_{X}e=0$.

-   $\frac{1}{n}\sum_{i=1}^{n}\widehat{e}_{i}=0$ if $x_{i}$ contains a
    constant.

Goodness of Fit
---------------

The so-called R-square is the most popular measure of goodness-of-fit in
the linear regression. R-square is well defined only when a constant is
included in the regressors. Let
$M_{\iota}=I_{n}-\frac{1}{n}\iota\iota'$, where $\iota$ is an $n\times1$
vector of 1’s. $M_{\iota}$ is the *demeaner*, in the sense that
$M_{\iota}\left(z_{1},\ldots,z_{n}\right)'=\left(z_{1}-\overline{z},\ldots,z_{n}-\overline{z}\right)'$,
where $\bar{z}=\frac{1}{n}\sum_{i=1}^{n}z_{i}$. For any $X$, we can
decompose $Y=P_{X}Y+M_{X}Y=\widehat{Y}+\widehat{e}$. The total variation
is
$$Y'M_{\iota}Y=\left(\widehat{Y}+\widehat{e}\right)'M_{\iota}\left(\widehat{Y}+\widehat{e}\right)=\widehat{Y}'M_{\iota}\widehat{Y}+2\widehat{Y}'M_{\iota}\widehat{e}+\widehat{e}'M_{\iota}\widehat{e}=\widehat{Y}'M_{\iota}\widehat{Y}+\widehat{e}'\widehat{e}$$
where the last equality follows by $M_{\iota}\widehat{e}=\widehat{e}$ as
$\frac{1}{n}\sum_{i=1}^{n}\widehat{e}_{i}=0$, and
$\widehat{Y}'\widehat{e}=Y'P_{X}M_{X}e=0$. R-square is defined as
$\widehat{Y}'M_{\iota}\widehat{Y}/Y'M_{\iota}Y$.

Frish-Waugh-Lovell Theorem
--------------------------

This theorem is the sample version of the subvector regression.

If $Y=X_{1}\beta_{1}+X_{2}\beta_{2}+e$, then
$\widehat{\beta}_{1}=\left(X_{1}'M_{X_{2}}X_{1}\right)^{-1}X_{1}'M_{X_{2}}Y.$

Statistical Properties of Least Squares
=======================================

To talk about the statistical properties in finite sample, we impose the
following assumptions.

1.  The data $\left(y_{i},x_{i}\right)_{i=1}^{n}$ is a random sample
    from the same data generating process $y_{i}=x_{i}'\beta+e_{i}$.

2.  $e_{i}|x_{i}\sim N\left(0,\sigma^{2}\right)$.

Maximum Likelihood Estimation[\*]{}
-----------------------------------

Under the normality assumption,
$y_{i}|x_{i}\sim N\left(x_{i}'\beta,\gamma\right)$, where
$\gamma=\sigma^{2}$. The *conditional* likelihood of observing a sample
$\left(y_{i},x_{i}\right)_{i=1}^{n}$ is
$$\prod_{i=1}^{n}\frac{1}{\sqrt{2\pi\gamma}}\exp\left(-\frac{1}{2\gamma}\left(y_{i}-x_{i}'\beta\right)^{2}\right),$$
and the (conditional) log-likelihood function is
$$L\left(\beta,\gamma\right)=-\frac{n}{2}\log2\pi-\frac{n}{2}\log\gamma-\frac{1}{2\gamma}\sum_{i=1}^{n}\left(y_{i}-x_{i}'\beta\right)^{2}.$$
Therefore, the maximum likelihood estimator (MLE) coincides with the OLS
estimator, and
$\widehat{\gamma}_{\mathrm{MLE}}=\widehat{e}'\widehat{e}/n$.

Finite Sample Distribution
--------------------------

We can show the finite-sample exact distribution of $\widehat{\beta}$.
*Finite sample distribution* means that the distribution holds for any
$n$; it is in contrast to *asymptotic distribution*, which holds only
when $n$ is arbitrarily large.

Since
$$\widehat{\beta}=\left(X'X\right)^{-1}X'y=\left(X'X\right)^{-1}X'\left(X'\beta+e\right)=\beta+\left(X'X\right)^{-1}X'e,$$
we have the estimator
$\widehat{\beta}|X\sim N\left(\beta,\sigma^{2}\left(X'X\right)^{-1}\right)$,
and
$$\widehat{\beta}_{k}|X\sim N\left(\beta_{k},\sigma^{2}\eta_{k}'\left(X'X\right)^{-1}\eta_{k}\right)\sim N\left(\beta_{k},\sigma^{2}\left(X'X\right)_{kk}^{-1}\right),$$
where $\eta_{k}=\left(1\left\{ l=k\right\} \right)_{l=1,\ldots,K}$ is
the selector of the $k$-th element.

In reality, $\sigma^{2}$ is an unknown parameter, and
$$s^{2}=\widehat{e}'\widehat{e}/\left(n-K\right)=e'M_{X}e/\left(n-K\right)$$
is an unbiased estimator of $\sigma^{2}$. Consider the $T$-statistic
$$T_{k}=\frac{\widehat{\beta}_{k}-\beta_{k}}{\sqrt{s^{2}\left[\left(X'X\right)^{-1}\right]_{kk}}}=\frac{\left(\widehat{\beta}_{k}-\beta_{k}\right)/\sqrt{\sigma^{2}\left[\left(X'X\right)^{-1}\right]_{kk}}}{\sqrt{\frac{e'}{\sigma}M_{X}\frac{e}{\sigma}/\left(n-K\right)}}.$$
The numerator follows a standard normal, and the denominator follows
$\frac{1}{n-K}\chi^{2}\left(n-K\right)$. Moreover, the numerator and the
denominator are independent. As a result, $T_{k}\sim t\left(n-K\right)$.

Mean and Variance
-----------------

Now we relax the normality assumption and statistical independence.
Instead, we assume a random sample and $$\begin{aligned}
y_{i} & = & x_{i}'\beta+e_{i}\nonumber \\
E[e_{i}|x_{i}] & = & 0\label{eq:mean_indep}\\
E\left[e_{i}^{2}|x_{i}\right] & = & \sigma^{2}.\label{eq:homo}\end{aligned}$$
(\[eq:mean\_indep\]) is the *mean independence* assumption, and
(\[eq:homo\]) is the *homoskedasticity* assumption.

(Heteroskedasticity) If $e_{i}=x_{i}u_{i}$, where $x_{i}$ is a scalar
random variable, $u_{i}$ is independent of $x_{i}$,
$E\left[u_{i}\right]=0$ and $E\left[u_{i}^{2}\right]=\sigma^{2}$. Then
$E\left[e_{i}|x_{i}\right]=0$ but
$E\left[e_{i}^{2}|x_{i}\right]=\sigma_{i}^{2}x_{i}^{2}$ is a function of
$x_{i}$. We say $e_{i}^{2}$ is a heteroskedastic error.

These assumptions are about the first and second moment of $e_{i}$
conditional on $x_{i}$. Unlike the normality assumption, they do not
restrict the entire distribution of $e_{i}$.

-   Unbiasedness:
    $$E\left[\widehat{\beta}|X\right]=E\left[\left(X'X\right)^{-1}XY|X\right]=E\left[\left(X'X\right)^{-1}X\left(X'\beta+e\right)|X\right]=\beta.$$
    Unbiasedness does not rely on homoskedasticity.

-   Variance: $$\begin{aligned}
    \mathrm{var}\left(\widehat{\beta}|X\right) & = & E\left[\left(\widehat{\beta}-E\widehat{\beta}\right)\left(\widehat{\beta}-E\widehat{\beta}\right)'|X\right]\\
     & = & E\left[\left(\widehat{\beta}-\beta\right)\left(\widehat{\beta}-\beta\right)'|X\right]\\
     & = & E\left[\left(X'X\right)^{-1}X'ee'X\left(X'X\right)^{-1}|X\right]\\
     & = & \left(X'X\right)^{-1}X'E\left[ee'|X\right]X\left(X'X\right)^{-1}\\
     & = & \left(X'X\right)^{-1}X'\left(\sigma^{2}I_{n}\right)X\left(X'X\right)^{-1}\\
     & = & \sigma^{2}\left(X'X\right)^{-1}.\end{aligned}$$

Gauss-Markov Theorem[\*]{}
--------------------------

Gauss-Markov theorem justifies the OLS estimator as the efficient
estimator among all linear unbiased ones. *Efficient* here means that it
enjoys the smallest variance in a family of estimators.

There are numerous linearly unbiased estimators. For example,
$\left(Z'X\right)^{-1}Z'y$ for $z_{i}=x_{i}^{2}$ is unbiased because
$E\left[\left(Z'X\right)^{-1}Z'y\right]=E\left[\left(Z'X\right)^{-1}Z'\left(X\beta+e\right)\right]=\beta$.

Let $\tilde{\beta}=A'y$ be a generic linear estimator, where $A$ is any
$n\times K$ functions of $X$. As
$$E\left[A'y|X\right]=E\left[A'\left(X\beta+e\right)|X\right]=A'X\beta.$$
So the linearity and unbiasedness of $\tilde{\beta}$ implies
$A'X=I_{n}$. Moreover, the variance
$$\mbox{var}\left(A'y|X\right)=E\left[\left(A'y-\beta\right)\left(A'y-\beta\right)'|X\right]=E\left[A'ee'A|X\right]=\sigma^{2}A'A.$$
Let $C=A-X\left(X'X\right)^{-1}.$ $$\begin{aligned}
 &  & A'A-\left(X'X\right)^{-1}\\
 & = & \left(C+X\left(X'X\right)^{-1}\right)'\left(C+X\left(X'X\right)^{-1}\right)-\left(X'X\right)^{-1}\\
 & = & C'C+\left(X'X\right)^{-1}X'C+C'X\left(X'X\right)^{-1}=C'C,\end{aligned}$$
where the last equality follows as
$$\left(X'X\right)^{-1}X'C=\left(X'X\right)^{-1}X'\left(A-X\left(X'X\right)^{-1}\right)=\left(X'X\right)^{-1}-\left(X'X\right)^{-1}=0.$$
Therefore $A'A-\left(X'X\right)^{-1}$ is a positive semi-definite
matrix. The variance of any $\tilde{\beta}$ is no smaller than the OLS
estimator $\widehat{\beta}$.

Homoskedasticity is a restrictive assumption. Under homoskedasticity,
$\mathrm{var}\left(\widehat{\beta}\right)=\sigma^{2}\left(X'X\right)^{-1}$.
Popular estimator of $\sigma^{2}$ is the sample mean of the residuals
$\widehat{\sigma}^{2}=\frac{1}{n}\widehat{e}'\widehat{e}$ or the
unbiased one $s^{2}=\frac{1}{n-K}\widehat{e}'\widehat{e}$. Under
heteroskedasticity, Gauss-Markov theorem does not apply.